In [2]:
import xarray as xr
import glob as glob
import os
import netCDF4 as nc
from rasterio.warp import reproject, Resampling, calculate_default_transform
import rasterio
import rioxarray

In [31]:
# Read in the data
year = '2014'
month = '01'
directory = f'../data/{year}/{month}/'
filename = f'cmems_obs-wind_glo_phy_my_l4_P1M_{year}{month}_clipped'
ds = xr.open_dataset(directory + filename + '.nc')
eastward_winds = ds.eastward_wind
northward_winds = ds.northward_wind
eastward_winds.rio.set_spatial_dims('lon', 'lat', inplace=True)
northward_winds.rio.set_spatial_dims('lon', 'lat', inplace=True)
crs = 'EPSG:4326'
eastward_winds.rio.set_crs(crs).rio.to_raster(directory + filename + '_eastward_winds.tif')
northward_winds.rio.set_crs(crs).rio.to_raster(directory + filename + '_northward_winds.tif')



In [34]:
import rasterio
file = '../data/2014/01/cmems_obs-wind_glo_phy_my_l4_P1M_201401_clipped_eastward_winds.tif'
#file = '../data/2014/01/2014_01_jplMURSST41_.tif'
# Abre el archivo raster
with rasterio.open(file) as src:
    # Obtiene el CRS
    crs = src.crs

print("CRS del archivo raster:", crs)


CRS del archivo raster: EPSG:4326


In [35]:
def reproj_match(infile, match, outfile):
    """Reproject a file to match the shape and projection of existing raster. 
    
    Parameters
    ----------
    infile : (string) path to input file to reproject
    match : (string) path to raster with desired shape and projection 
    outfile : (string) path to output file tif
    """
    # open input
    with rasterio.open(infile) as src:
        src_transform = src.transform
        
        # open input to match
        with rasterio.open(match) as match:
            dst_crs = match.crs
            
            # calculate the output transform matrix
            dst_transform, dst_width, dst_height = calculate_default_transform(
                src.crs,     # input CRS
                dst_crs,     # output CRS
                match.width,   # input width
                match.height,  # input height 
                *match.bounds,  # unpacks input outer boundaries (left, bottom, right, top)
            )

        # set properties for output
        dst_kwargs = src.meta.copy()
        dst_kwargs.update({"crs": dst_crs,
                           "transform": dst_transform,
                           "width": dst_width,
                           "height": dst_height,
                           "nodata": 0})
        print("Coregistered to shape:", dst_height,dst_width,'\n Affine',dst_transform)
        # open output
        with rasterio.open(outfile, "w", **dst_kwargs) as dst:
            # iterate through bands and write using reproject function
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=dst_transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
                

In [39]:
# reproject sst and chla to match the shape and projection of the winds

sst_file = f'{directory}/{year}_{month}_jplMURSST41_.tif'
chla_file = f'{directory}/{year}_{month}_NESDIS_VHNSQ_chla.tif'
match_file = f'{directory}/{filename}_eastward_winds.tif'
out_sst = f'{sst_file}_reproj.tif'
out_chla = f'{chla_file}_reproj.tif'

reproj_match(sst_file, match_file, out_sst)
reproj_match(chla_file, match_file, out_chla)

Coregistered to shape: 128 212 
 Affine | 0.25, 0.00,-163.00|
| 0.00,-0.25, 55.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 128 212 
 Affine | 0.25, 0.00,-163.00|
| 0.00,-0.25, 55.00|
| 0.00, 0.00, 1.00|
